# Assignment 3

***

*Consensus algorithms*

***

In this practical work, we will implement the details of both Proof-of-Stake (PoS) and as a bonus Proof-of-Work (PoW). Since PoS is becoming the most popular consensus algorithm, and suits any decentralized system well, it is the most interesting to look after.

It may seem counter-intuitive since until now we haven't seen nor coded a decentralized system (this is for Assignment 4). We will stick to one central blockchain on which multiple public keys operate.

So one would argue : why doing it now since once we port the model to a decentralized network, blockchain instances will be out of synchronization. Well the answer, as we've seen in the lecture, is **deterministicity**. The consensus is **universal** and does not need people to communicate. Like a law of nature, somehow.

Take as an example the pi constant ```𝜋 ≈ 3.14159...```. Since we've all been taught the same mathematics in class, we all learnt the same operators ```+```, ```-```, ```÷``` ..., the traditional digits 0..9, and all the rest, which are all really just a human construct, we are all able to calculate ```𝜋``` (we actually did in Assignment 1). We can thus say there is a universal consensus on ```𝜋```. You do not need to talk with a libanese student to agree on the value of ```𝜋```.

For blockchains, the same applies. Every node in the network owns the same software that runs the blockchain (like us, the same mathematics) and can thus **calculate the same forger for next block**. This way, a consensus is reached without exchanging information.

In [3]:
%load_ext autoreload


In [4]:
# Mandatory cell, please execute it

%autoreload 2

from sys import path

path.append('../scripts')

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

### Staking

In Proof-of-Stake, there is "Stake" so we need the possibility to stake tokens.

In the context of blockchains, "staking" refers to the action of depositing a fixed amount of tokens to strengthen one's voting rights in important decisions concerning the blockchain. The primary of these decisions is the selection of the next forger, but each blockchain project can include additional decisions, such as the project's direction, reward choices, and more...

As we haven't implemented an economic system in our blockchain (our blockchain is purely educational), we will assume that every actor in the blockchain has an infinite amount of tokens in their pocket. Therefore, you can stake any amount you want for each wallet.

Now, we need to code the staking operation.

***

<font color="7777aa">In your "scripts" folder, create a file called `stake.py` in which you must implement the `StakingOperation` class. For now, just code its constructor, nothing more.</font>

This class should adhere to the following model, as we've just discussed:
* A staking operation involves a person and an amount of tokens to be staked (`tokenAmount`).
* These operations will be stored in the blockchain (and therefore gain all the properties of security, immutability, etc...)

> <details><summary><strong>Click here to show help 1</strong></summary>The description I provided should strongly remind you of another data structure you saw in Assignment 2.</details>

> <details><summary><strong>Click here to show help 2</strong></summary><code>StakingOperation</code> inherits from <code>Certificate</code>, you just need to add the <code>tokenAmount</code> field in the class and its constructor.</details>

In [5]:
from stake import StakingOperation

dummyStakingOperation = StakingOperation('9cbe2a6713', 42)

assert dummyStakingOperation.issuerPublicKey == '9cbe2a6713'   # Has an issuer
assert dummyStakingOperation.tokenAmount == 42                 # Staking 42 tokens

'Success!'

'Success!'

***
<font color="7777aa">In the same way you did it for the `Block` class in Assignment 2, rewrite the `build_payload()` function to include the additional data.</font>

> <details><summary><strong>Click here to show help</strong></summary>You simply need to add <code>tokenAmount</code> to it.</details>

In [6]:
dummyStakingOperation.build_payload()

{'issuerPublicKey': '9cbe2a6713',
 'timestamp': 1738830597584,
 'tokenAmount': 42}

You can now use your blockchain to perform staking operations, and the following cell demonstrates how to do it.

For the entire duration of the Assignment, we will have the following setup:

* Two actors (**Alice** and **Bob**) represented by their wallets (`walletAlice` and `walletBob`).
* One and only one **blockchain** (`blockchainAssignment3`) on which we will add staking operations and general certificates.

Let's start with Alice and Bob staking the following amounts:
* Alice: 4
* Bob: 2.5

In [7]:
from wallet import Wallet
from block import Block
from blockchain import Blockchain

# Two actors

walletAlice = Wallet()
walletBob = Wallet()

def reset_staking_blockchain():

    global walletAlice, walletBob, blockchainAssignment3, stakingAlice1, stakingBob1, block1
    
    # Our blockchain
    
    blockchainAssignment3 = Blockchain()
    
    # Alice wants to stake 4
    
    stakingAlice1 = StakingOperation(walletAlice.publicKey, 4)
    walletAlice.sign(stakingAlice1)
    
    # Bob wants to stake 2.5
    
    stakingBob1 = StakingOperation(walletBob.publicKey, 2.5)
    walletBob.sign(stakingBob1)
    
    # To ensure both staking operations are inside the blockchain, Alice creates a block, put them in it and adds the block to the blockchain.
    
    block1 = Block(walletAlice.publicKey, 1, blockchainAssignment3.get_latest_block().hash(), [stakingAlice1, stakingBob1])
    walletAlice.sign(block1)
    blockchainAssignment3.blockList.append(block1)

reset_staking_blockchain()

# If you did bonus 1 of Assignment 2 (or if you downloaded the solution), take a peek at your blockchain :

if not hasattr(blockchainAssignment3, 'display'):
    raise StopExecution
    
blockchainAssignment3.display()

Issuer public key: 000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Signature: None
Timestamp: 1738830598385
Hash: 1ede4baa242162337ecb4f3730b3ee4ce74f980ac614c747ee698496bd7714eb
Is legit: True

Index in blockchain: 0
Parent block hash: 0000000000000000000000000000000000000000000000000000000000000000
Certificate list:


Issuer public key: 30820122300d06092a864886f70d01010105000382010f003082010a0282010100c4628604d2a52b75bd286f2df5d240ee2e4d5960d1d4520de26b

It would be interesting to code a function that allows us to determine the total staked tokens for each actor. This will be particularly useful for the consensus algorithm.

The idea is simple: we will traverse the entire blockchain and calculate, using the staking operations found within it, the total staked tokens for everyone.

Since this function only deals with staking operations, we will write it directly in the `StakingOperation` class.

***

You will need to write a static function. In Python, you can do this by removing `self` from the function's arguments and adding the `@staticmethod` attribute. Here's an example:

```python
class MyClass:

    @staticmethod
    def my_static_function(arguments):
        pass
```

To use it, simply call `MyClass.my_static_function(arguments)`.

***

<font color="7777aa">In your `StakingOperation` class, write the static function `build_staking_accounts(blockList)` that takes as input **any list of blocks** (not a blockchain) and returns a Python dictionary. The keys of this dictionary should be the public keys of the people who have staked tokens in this list of blocks, and the values should be the amount of tokens staked by these individuals.</font>

> `build_staking_accounts(blockList)` takes a list of blocks as a parameter rather than a blockchain because it's a function that can be used on a subset of blocks from any blockchain. In general, since this function only deals with a list of blocks, there's no need to complicate the input parameter.

> <details><summary><strong>Click here to show help 1</strong></summary>You can use the built-in <code>isinstance(object, class)</code> function as demonstrated in the cell below.</details>

> <details><summary><strong>Click here to show help 2</strong></summary>Create an empty dictionary, which will be your output dictionary. Then, iterate through each block in the list and each certificate in those blocks. For each certificate found, check if it's indeed a <code>StakingOperation</code>, and if so, add the amount to your output dictionary.</details>

In [8]:
from certificate import Certificate

reset_staking_blockchain()

# We'll add a second block containing a staking operation as well as an arbitrary certificate

if len(blockchainAssignment3.blockList) == 2:
    
    # Arbitrary certificate of Bob
    
    certificateBob1 = Certificate(walletBob.publicKey)
    walletBob.sign(certificateBob1)
    
    # Alice eventually decides to get 1 token back
    
    stakingAlice2 = StakingOperation(walletAlice.publicKey, -1)
    walletAlice.sign(stakingAlice2)
    
    # New block to engrave those certificates
    
    block2 = Block(walletBob.publicKey, 2, blockchainAssignment3.get_latest_block().hash(), [certificateBob1, stakingAlice2])
    walletBob.sign(block2)
    blockchainAssignment3.blockList.append(block2)
    
# Let's calculate the stakings
accountsAfterBlock2 = StakingOperation.build_staking_accounts(blockchainAssignment3.blockList)

assert isinstance(accountsAfterBlock2, dict)              # I asked for a dictionary
assert len(accountsAfterBlock2) == 2                      # 2 actors have staked
assert accountsAfterBlock2[walletAlice.publicKey] == 3    # Alice staked 3
assert accountsAfterBlock2[walletBob.publicKey] == 2.5    # Bob staked 2.5

"Success!"

'Success!'

### Proof-of-Stake

Remember what we learned in class: in a blockchain, different actors agree on the identity of the entity that has the right to forge the next block to be added. This is called the **consensus algorithm**.

Now it's time to implement the ***Proof-of-Stake*** consensus algorithm for our blockchain.

The idea is quite simple, as we discussed in class; it's a system similar to a lottery or a raffle:

* First, we calculate the staking accounts for all actors who have staked (by traversing the blockchain up to this point).
* For each actor who has staked, we will assign them as many raffle tickets as they have staked tokens (rounded down to the nearest whole number because we can have fractional tokens...).
* Next, we need to decide who has won. To do this, we hash the last block of the blockchain, which we then call `forgeHash`.
* Finally, we hash all the raffle tickets. The ticket with the hash closest in numerical distance to `forgeHash` is declared the winner, and its owner gains the right to forge the next block (to get the numerical distance, we convert the hashes to `int`).

This algorithm works because, since the blockchain is the same for all network actors, each one can perform these calculations on their own and arrive at the same winner as the others (as explained at the beginning of this notebook).

Let's start by modeling a raffle ticket.

***

<font color="7777aa">In your "scripts" folder, create a file called `ticket.py` where you will implement the `RaffleTicket` class. This class should have three public fields: its owner `ownerPublicKey`, its ticket number `number`, and the hash of the raffle it participates in `raffleHash`.</font>

In [9]:
from ticket import RaffleTicket

dummyTicket = RaffleTicket(walletAlice.publicKey, 1, blockchainAssignment3.get_latest_block().hash())

assert dummyTicket.ownerPublicKey == walletAlice.publicKey                 # Owner is Alice
assert dummyTicket.number == 1                                             # Ticket number is 1
assert dummyTicket.raffleHash == blockchainAssignment3.get_latest_block().hash()   # Raffle hash is the hash of latest block

'Success!'

'Success!'

<font color="7777aa">Add a `hash()` function to this class that transforms the ticket into a hash **in hexadecimal form** (it's better to keep our standard format and convert it to an integer later).

**Attention! Remember that your hash must be identical if you have exactly the same data in the ticket...**</font>

> <details><summary><strong>Click here to show help 1</strong></summary>You have already done this process in Assignment 1, the pattern remains the same. If your <code>build_payload()</code> function returns a dictionary, for example, don't forget to sort the keys in alphabetical order before hashing (<code>dumps(payload, sort_keys=True)</code>).</details>

> <details><summary><strong>Click here to show help 2</strong></summary>Convert the ticket into a string and then use the <code>cryptography.hash_string(string)</code> function.</details>

> <details><summary><strong>Click here to show help 3</strong></summary>To transform the ticket into a string, take inspiration from the payload: extract the values of the important fields (here <code>ownerPublicKey</code>, <code>number</code>, and <code>raffleHash</code> are relevant), and apply the same process you used for Assignment 2.</details>

In [10]:
dummyTicket.hash()

'32edf7f094cbf8b8f30b75129cd937a0047623f0398e998b312fdbe5de67a19c'

Now that we are able to distribute raffle tickets to our stakers, we can implement the Proof-of-Stake algorithm.

***

<font color="7777aa">Create a file called `proof_of_stake.py` (still in the "scripts" folder). Since many consensus algorithms exist, and to stay true to the fundamental principles of Object-Oriented Programming, you should create a `ProofOfStake` class that we will use as a consensus object at the `Node` level (Assignment 4).

**What should be in its constructor?**

This question is very interesting. When we create the blockchain, it only contains the genesis block. So, since no one has staked tokens yet, and staking must be done for a block to be considered, how can we forge block number 1? No raffle tickets can be distributed...

In fact, an effective solution would be to have a **default forger**, someone capable of forging if no one has staked. Therefore, in the constructor of `ProofOfStake`, we need a default forger `defaultForgerPublicKey`.</font>

In [11]:
from proof_of_stake import ProofOfStake

dummyProofOfStake = ProofOfStake(walletAlice.publicKey)

assert dummyProofOfStake.defaultForgerPublicKey == walletAlice.publicKey      # Default forger is Alice

'Success!'

'Success!'

Before we move on to coding the algorithm described with the raffle analogy, let's take the time to implement a small utility function that calculates the distance between two hashes **in their hexadecimal form**.

***

<font color="7777aa">In your `ProofOfStake` class, implement the `hash_distance(hash1, hash2)` function that returns, as an integer, the integer distance between the two hashes `hash1` and `hash2` **in their hexadecimal form**.

Your function does not need to be static. Keep it simple.</font>

> <details><summary><strong>Click here to show help 1</strong></summary>To convert a hexadecimal <code>string</code> to an <code>int</code>, use the <code>int(string, 16)</code> function.</details>

> <details><summary><strong>Click here to show help 2</strong></summary>The integer distance between two integers <code>a</code> and <code>b</code> is calculated with <code>abs(a - b)</code>.</details>

In [12]:
hash1 = '67c5ba7'
hash2 = 'ff837ea'

assert dummyProofOfStake.hash_distance(hash1, hash2) == 159112259

"Success!"

'Success!'

Remember what we learned in class: **a consensus algorithm is a function that takes as input a list of blocks and the next block to be added to this list, and checks if this block has a valid forger.** This is exactly what we are going to do here.

In Proof-of-Stake, **the next block has a valid forger if that forger has won the raffle.** To create the consensus function, we first need to create a function that tells us who wins the raffle.

***

<font color="7777aa">Now, implement the `get_next_forger_public_key(blockList)` function, which, given any list of blocks, provides the public key of the forger of the next block to be added to this list (and therefore the raffle winner).

Please read the algorithm description at the beginning of the **Proof-of-Stake** section carefully.

Don't forget to consider the default forger.</font>

> `get_next_forger_public_key(blockList)` also takes a list of blocks as a parameter, for the same reasons mentioned earlier for `build_stake_accounts(blockList)`.

> <details><summary><strong>Click here to show help 1</strong></summary>To round a number down to the nearest whole number, use the built-in function <code>int(object)</code>.</details>

> <details><summary><strong>Click here to show help 2</strong></summary>This is simply an algorithm that calculates the minimum distance between all the tickets and the hash of the last block in the blockchain.</details>

> <details><summary><strong>Click here to show help 3</strong></summary>Start by using the <code>build_staking_accounts(blockchain)</code> function, and then allocate the tickets based on what it provides. This means that for each user in this dictionary, you need to create as many tickets as the tokens they have staked. Then, find the closest ticket using the <code>hash_distance(hash1, hash2)</code> function, which will give you the winner.</details>

***

**The following 2 cells must execute without errors.**

In [13]:
# Verifying default forger

emptyBlockchain = Blockchain()

assert dummyProofOfStake.get_next_forger_public_key(emptyBlockchain.blockList) == walletAlice.publicKey

"Success!"

'Success!'

In [14]:
# Is next forger Alice or Bob ?

nextForger = dummyProofOfStake.get_next_forger_public_key(blockchainAssignment3.blockList)

if nextForger == walletAlice.publicKey:
    print('Alice is the next forger')
else:
    print('Bob is the next forger')

Alice is the next forger


<font color="7777aa">Now, create the `is_next_block_forger_legit(blockList, nextBlock)` function that returns `True` if the issuer of the next block `nextBlock` is indeed the winner of the raffle of `blockList`, and `False` otherwise.</font>

> <details><summary><strong>Click here to show help</strong></summary>Use the function you just coded.</details>

In [15]:
nextBlock = Block(nextForger, len(blockchainAssignment3.blockList), blockchainAssignment3.get_latest_block().hash(), [])

assert dummyProofOfStake.is_next_block_forger_legit(blockchainAssignment3.blockList, nextBlock)

"Success!"

'Success!'

So now we have a tool that allows us to know if a new block to be added to our blockchain has been forged by the only person entitled to forge the next block (simply use the `is_next_block_forger_legit(blockList, nextBlock)` function to see if this block was forged by the right person).

As capable as we are of preventing new blocks from being forged by the wrong people, if someone sends us a complete blockchain, we cannot yet attest to its validity.

Let's say a fraudulent actor on the network decides to completely change the blockchain by becoming the owner of all its blocks. Then they send it to you because you've just joined the network and need to retrieve all the blockchain data to be up to date. From your point of view, since they have signed all the blocks, their blockchain is legal. However, this is not the case at all because they are not entitled to forge all the blocks (or the probability is very low).

Remember in Assignment 2, we wrote the `is_legit()` function that validates a blockchain. It's time to also take into account the forger's identity in the validation.

Since several consensus algorithms exist, if you change them tomorrow (for example, switch to Proof-of-Work), your code must adapt. For this reason, we will implement an `is_blockchain_legit(blockchain)` function directly in the `ProofOfStake` class.

***

<font color="7777aa">In the `ProofOfStake` class, implement the `is_blockchain_legit(blockchain)` function. This function should check, for each block in the blockchain, if its forger is indeed the expected forger given the previous block.

If any single block has a forger who did not win the raffle, this function should return `False`, otherwise it returns `True`.</font>

> <details><summary><strong>Click here to show help 1</strong></summary>Be careful with block index 0 (the first block)!!! Its forger is the black hole.</details>

> <details><summary><strong>Click here to show help 2</strong></summary>Iterate through the list of blocks <code>blockchain.blockList</code> in order, and for each block, verify that it was forged by the right person given the previous blocks (what do you put in <code>blockList</code> and <code>nextBlock</code> in the <code>is_next_block_forger_legit(blockList, nextBlock)</code> function?).</details>

In [16]:
block2 = blockchainAssignment3.blockList[2]
block2LegitForger = dummyProofOfStake.get_next_forger_public_key(blockchainAssignment3.blockList[:-1])
block2.issuerPublicKey = block2LegitForger

if block2LegitForger == walletAlice.publicKey:
    walletAlice.sign(block2)
else:
    walletBob.sign(block2)

assert dummyProofOfStake.is_blockchain_legit(blockchainAssignment3)

if block2LegitForger == walletAlice.publicKey:
    block2.issuerPublicKey = walletBob.publicKey
    walletBob.sign(block2)
else:
    block2.issuerPublicKey = walletAlice.publicKey
    walletAlice.sign(block2)

assert not dummyProofOfStake.is_blockchain_legit(blockchainAssignment3)

"Assignment completed! Congratz!"

'Assignment completed! Congratz!'

### Bonus 1 - Monte Carlo (Episode 2)

In our current configuration, Alice has staked 3 tokens, and Bob has staked 2.5 tokens. For our consensus algorithm, this means that Alice has 60% of the decision-making power (3 tickets out of 5), and Bob has 40% (2 tickets out of 5).

Let's verify if this is indeed the case with your implementation. We will use the same method as in Assignment 1: the Monte Carlo method. Let's simulate 10,000 raffles and observe the probabilities of Alice and Bob winning.

***

<font color="7777aa">Complete the following cell to simulate 10,000 raffles. Ensure that the hash of the last block in the `blockchainAssignment3` blockchain changes with each simulation (to vary the raffles).</font>

In [17]:
import random


nSimulations = 100

winsAlice = 0
winsBob = 0

reset_staking_blockchain()

stackingAccounts = StakingOperation.build_staking_accounts(blockchainAssignment3.blockList)
# The espected porcentage of wins
aliceStaking = int( stackingAccounts[walletAlice.publicKey])
bobStaking = int(stackingAccounts[walletBob.publicKey])
expectedAlice = aliceStaking / (aliceStaking + bobStaking)
expectedBob = bobStaking / (aliceStaking + bobStaking)
print(f'Expected Alice: {expectedAlice}')
print(f'Expected Bob: {expectedBob}')

for _ in range(nSimulations):
    
    nextForger = dummyProofOfStake.get_next_forger_public_key(blockchainAssignment3.blockList)
    nonce = random.randint(0, 1000000)
    nextBlock = Block(nextForger, len(blockchainAssignment3.blockList), blockchainAssignment3.get_latest_block().hash(), [], nonce)
    
    if nextForger == walletAlice.publicKey:
        walletAlice.sign(nextBlock)
        winsAlice += 1
    elif nextForger == walletBob.publicKey:
        walletBob.sign(nextBlock)
        winsBob += 1
    
    blockchainAssignment3.blockList.append(nextBlock)
    
        
print(f'Alice has a {int(round(winsAlice / nSimulations, 2) * 100)}% chance to forge')
print(f'Bob has a {int(round(winsBob / nSimulations, 2) * 100)}% chance to forge')

Expected Alice: 0.6666666666666666
Expected Bob: 0.3333333333333333
Alice has a 61% chance to forge
Bob has a 39% chance to forge


### Bonus 2 - Proof-of-Work

You've learned all about Proof-of-Work in the course. For this bonus, I'm asking you to implement the basic mechanics of this algorithm (we won't change the puzzle difficulty based on forging speed).

***

<font color="7777aa">Add the `nonce` field to your `Block` class. This new field should be considered in all blockchain mechanisms. **Do not add it to the constructor parameters; simply initialize it to 0**.</font>

In [18]:
block = Block(walletAlice.publicKey, 0, 'void', [])

assert 'nonce' in block.__dict__

"Success!"

'Success!'

Let's start by providing ourselves with the means to solve the puzzle of this consensus algorithm. Just like we designed the `ProofOfStake` class, we will create the `ProofOfWork` class.

***

<font color="7777aa">Create the `ProofOfWork` class in a file named `proof_of_work.py`. Its constructor takes an integer `difficulty` as a parameter, which represents the number of leading zeros required to solve the puzzle.</font>

In [21]:
from proof_of_work import ProofOfWork

dummyProofOfWork = ProofOfWork(3)

assert dummyProofOfWork.difficulty == 3

"Success!"

'Success!'

<font color="7777aa">Add the `solve_puzzle(block)` function to it, which finds a nonce sufficient for the hash of the `block` to begin with at least as many zeros as indicated by `difficulty`. **This function then applies this nonce to the block**.</font>

In [22]:
dummyProofOfWork.solve_puzzle(block)

assert block.hash()[:dummyProofOfWork.difficulty] == '0' * dummyProofOfWork.difficulty

f"Success for a nonce of {block.nonce} (hash = {block.hash()})"

Try 5119 - 0000cc6865555a9cf599b39174e420b773b4f46d8c0084efb76335efea726b1e


'Success for a nonce of 4946244 (hash = 0000cc6865555a9cf599b39174e420b773b4f46d8c0084efb76335efea726b1e)'

<font color="7777aa">To maintain consistency with the interface of the `ProofOfStake` class, this class should have two functions: `is_next_block_forger_legit(blockList, nextBlock)` and `is_blockchain_legit(blockchain)`. Implement them.</font>

In [68]:
powBlockchain = Blockchain()

powBlock1 = Block(walletAlice.publicKey, 1, powBlockchain.get_latest_block().hash(), [])
walletAlice.sign(powBlock1)
dummyProofOfWork.solve_puzzle(powBlock1)
powBlockchain.blockList.append(powBlock1)

powBlock2 = Block(walletBob.publicKey, 2, powBlockchain.get_latest_block().hash(), [])
walletBob.sign(powBlock2)
dummyProofOfWork.solve_puzzle(powBlock2)

assert dummyProofOfWork.is_next_block_forger_legit(powBlockchain.blockList, powBlock2)

powBlockchain.blockList.append(powBlock2)

assert dummyProofOfWork.is_blockchain_legit(powBlockchain)

powBlock1.nonce -= 1

assert not dummyProofOfWork.is_blockchain_legit(powBlockchain)

"Your PoW implementation seems to work. Congratz!"

'Your PoW implementation seems to work. Congratz!'